# RaspBerry Pi Setup

## References

### Training the model  

1. Following [*TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi*](https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi)
   - Using python with OpenCV with the new libcamera stack (instead of legacy camera) needs picamera2, [installing it through here](https://www.raspberrypi.com/news/using-the-picamera2-library-with-tensorflow-lite/):
     - Installing the Picamera2 library:
       - Besides all requiered commands, run these: 
         ```bash
          sudo apt-get install libyaml-dev
          sudo apt-get install libudev-dev
          sudo apt install -y cmake
          sudo apt-get install python-jinja2
          sudo apt-get install python-ply
          pip install pyyaml
          sudo apt-get install python3-sphinx
          sudo apt-get install qttools5-dev-tools
         ```
       - and install all the dependencies in *README.rst* after running `git clone --branch picamera2 https://github.com/raspberrypi/libcamera.git && cd libcamera`
       - you might have to [increase the swap](https://stackoverflow.com/questions/30887143/make-j-8-g-internal-compiler-error-killed-program-cc1plus) to build it successfully too
   - notes:
     - [opencv with raspberry libcamera in C++](https://github.com/Qengineering/Libcamera-OpenCV-RPi-Bullseye-32OS)